In [1]:
#  Note, deleted the minimum-JSON-for-notebook-detection cell

We ran two cells before that gave us a `model` error

```python
# === CIFAR-10 bootstrap (one cell) ============================================
import os, json, shutil, random, sys
from pathlib import Path
from datetime import datetime

# ---- runtime knobs (quiet + threads) ----------------------------------------
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")
os.environ.setdefault("OMP_NUM_THREADS", "4")
os.environ.setdefault("TF_NUM_INTEROP_THREADS", "2")
os.environ.setdefault("TF_NUM_INTRAOP_THREADS", "4")

# ---- resolve TAGDIR (project tag root) ---------------------------------------
# If running from p_01/notebooks, TAGDIR is parent; else use CWD or $TAGDIR.
_notebooks = Path.cwd().name.lower() == "notebooks"
tagdir = Path(os.environ.get("TAGDIR") or (Path.cwd().parent if _notebooks else Path.cwd()))
os.environ["TAGDIR"] = str(tagdir)

# ---- project paths -----------------------------------------------------------
outdir = tagdir / "outputs"
csv_dir = outdir / "csv_logs"
gradcam_dir = outdir / "gradcam_images"
for d in (outdir, csv_dir, gradcam_dir):
    d.mkdir(parents=True, exist_ok=True)

# ---- Keras dataset cache scoped to project -----------------------------------
# This keeps CIFAR-10 under: <TAGDIR>/datasets
os.environ.setdefault("KERAS_HOME", str(tagdir))
proj_cache = Path(os.environ["KERAS_HOME"]) / "datasets"
proj_cache.mkdir(parents=True, exist_ok=True)

# Seed from ~/.keras/datasets if available and project cache is empty
home_cache = Path.home() / ".keras" / "datasets"
candidates = [("cifar-10-python.tar.gz", True), ("cifar-10-batches-py", False)]
if home_cache.exists() and not any((proj_cache / n).exists() for n, _ in candidates):
    for name, is_file in candidates:
        src = home_cache / name
        if src.exists():
            dst = proj_cache / src.name
            (shutil.copy2 if is_file else shutil.copytree)(src, dst, dirs_exist_ok=True)

# ---- seeding -----------------------------------------------------------------
SEED = int(os.environ.get("SEED", "137"))
random.seed(SEED)
try:
    import numpy as _np
    _np.random.seed(SEED)
except Exception:
    pass
try:
    import tensorflow as _tf
    _tf.random.set_seed(SEED)
except Exception:
    _tf = None  # Training may still run with torch; this cell is TF-friendly but not TF-required.

# ---- helpers for file naming & logging ---------------------------------------
def _ts():
    return datetime.now().strftime("%Y%m%dT%H%M%S")

def make_paths(seed: int = SEED):
    stamp = _ts()
    csv_path = csv_dir / f"train_history_seed{seed}_{stamp}.csv"
    json_path = outdir / f"test_summary_seed{seed}_{stamp}.json"
    return csv_path, json_path

def log_test_summary(test_acc: float, loss: float | None = None, seed: int = SEED, extra: dict | None = None):
    """
    Write the minimal JSON required by Q&R and print the sentinel line.
    Usage at end of eval:
        log_test_summary(test_acc, loss=float(test_loss))
    """
    _, json_path = make_paths(seed)
    payload = {"seed": seed, "test_acc": float(test_acc)}
    if loss is not None:
        payload["test_loss"] = float(loss)
    if extra:
        payload.update(extra)
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(payload, f, indent=2)
    print(f"[DONE] test_acc={payload['test_acc']:.4f}  ->  wrote {json_path}")

# ---- pretty banner -----------------------------------------------------------
print(
    "Bootstrap ready\n"
    f"  TAGDIR:         {tagdir}\n"
    f"  KERAS_HOME:     {Path(os.environ['KERAS_HOME'])}\n"
    f"  Outputs:        {outdir}\n"
    f"  CSV logs:       {csv_dir}\n"
    f"  Grad-CAM imgs:  {gradcam_dir}\n"
    f"  SEED:           {SEED}\n"
    f"  TensorFlow:     {('OK ' + _tf.__version__) if _tf else 'not imported'}"
)
# ============================================================================== 
```

and the second cell

```python
import sys
from pathlib import Path

tagdir = Path(os.environ["TAGDIR"])          # .../test_project_bash/p_01
pkg_parent = tagdir.parent                   # .../test_project_bash

# allow BOTH styles:
# 1) from scripts.py_utils_p_01 import ...
if str(tagdir) not in sys.path:
    sys.path.insert(0, str(tagdir))

# 2) from p_01.scripts.py_utils_p_01 import ...
if str(pkg_parent) not in sys.path:
    sys.path.insert(0, str(pkg_parent))

from p_01.scripts.py_utils_p_01 import log_test_summary, make_paths

test_loss, test_acc = model.evaluate(test_ds, verbose=0)
```

In [8]:
# ==== Minimal CIFAR-10 train+test with build_model (Q&R ready) ==============
import numpy as np, tensorflow as tf
from tensorflow.keras import layers, Model

# --- your build_model from earlier ------------------------------------------
def build_model(input_shape=(32, 32, 3), n_classes: int = 10) -> Model:
    """Tiny A0-style CNN with named layers; logits output (no softmax layer)."""
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, 3, padding="same", activation="relu", name="conv1")(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, padding="same", activation="relu", name="conv2")(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, padding="same", activation="relu", name="conv3")(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(n_classes, name="logits")(x)
    return Model(inputs, outputs, name="A0_CNN")

# --- seed & data ------------------------------------------------------------
SEED = 137
tf.random.set_seed(SEED)
np.random.seed(SEED)

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train.astype("float32")/255.0, x_test.astype("float32")/255.0

BATCH, EPOCHS = 128, 4
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000, seed=SEED).batch(BATCH)
test_ds  = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH)

# --- model build/compile -----------------------------------------------------
model = build_model()
model.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

# --- training ---------------------------------------------------------------
history = model.fit(train_ds, epochs=EPOCHS, verbose=2)

# --- evaluation + Q&R logging ----------------------------------------------
from scripts.py_utils_p_01 import log_test_summary, make_paths
test_loss, test_acc = model.evaluate(test_ds, verbose=0)
log_test_summary(test_acc, loss=float(test_loss),
                 seed=SEED,
                 extra={"epochs": EPOCHS, "batch": BATCH})

Epoch 1/4
391/391 - 112s - loss: 1.6037 - accuracy: 0.4191 - 112s/epoch - 285ms/step
Epoch 2/4
391/391 - 114s - loss: 1.1891 - accuracy: 0.5791 - 114s/epoch - 292ms/step
Epoch 3/4
391/391 - 123s - loss: 1.0119 - accuracy: 0.6409 - 123s/epoch - 313ms/step
Epoch 4/4
391/391 - 121s - loss: 0.8989 - accuracy: 0.6861 - 121s/epoch - 310ms/step
[DONE] test_acc=0.6686  ->  wrote /home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_01/notebooks/outputs/test_summary_seed137_20250906T125134.json


PosixPath('/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_01/notebooks/outputs/test_summary_seed137_20250906T125134.json')